##What are Convolutions?

ここでは、コンボリューションを使用してコンピュータビジョンを強化する方法を探ります。しかし、コンボリューションとは何でしょうか？ここでは、コンボリューションとは何か、どのように動作するのかを探求し、つぎの章では、ニューラルネットワークでの使用方法を見ていきます。



畳み込みと一緒に、画像を圧縮して特徴をさらに強調する「プーリング」と呼ばれるものを使用します。ここでは、プーリングがどのように機能するかも見てみましょう。

##Limitations of the previous DNN


前回のラボでは、Fashion MNISTデータセットを用いてファッションアイテムの画像分類器を訓練する方法を見ました。これでかなり正確な分類器が得られましたが、明らかな制約がありました：画像は28x28、グレースケールで、アイテムは画像の中心にあることです。

例えば、以下はFashion MNISTの画像の一部です。
![セーターとブーツの写真](https://cdn-images-1.medium.com/max/1600/1*FekMt6abfFFAFzhQcnjxZg.png)

あなたが作成したDNNは、単に生のピクセルからセーターを構成するものと、ブーツを構成するものを学習しました。しかし、この画像をどのように分類するか考えてみましょう。

![ブーツの画像](https://cdn.pixabay.com/photo/2013/09/12/19/57/boots-181744_1280.jpg)

この画像にブーツがあることは明らかですが、分類器はいくつかの理由で失敗します。第一に、もちろん、これは28x28のグレースケールではありません。より重要なことは、分類器は左向きのブーツの画像で訓練されており、ブーツが何であるかということで訓練されていないということです。

そこでコンボリューションが非常に強力なのです。畳み込みとは、画像を処理し、画像の共通性を示す特徴を抽出するフィルタです。このラボでは、コンボリューションがどのように動作するかを見ますが、画像を処理して、画像から特徴を抽出できるかどうかを見てみましょう!






畳み込みを実行するのはとても単純です。単純に画像の各画素をスキャンして、その画素に隣接する画素を見るだけです。そしてこれらのピクセルの値にフィルタの等価な重みを掛け合わせます。

例えば次のように考えてみてください。

![画像上のコンボリューション](https://storage.googleapis.com/laurencemoroney-blog.appspot.com/MLColabImages/lab3-fig1.png)

![](https://techblog.gmo-ap.jp/wp-content/uploads/2019/04/3D_Convolution_Animation.gif)

この場合は 3x3 コンボリューションが指定されています。

現在の画素の値は 192 ですが、隣接する画素の値を見てフィルタで指定した値を掛け合わせて新しい画素の値を計算し、その値を最終的な値とします。


2Dのグレースケール画像に基本的な畳み込みを作成することで、畳み込みがどのように機能するかを探ってみましょう。まず、scipyの'asccent'画像を使って画像をロードします。これは、角度や線がたくさん入った素敵な組み込み画像です。


まずは、Pythonのライブラリをインポートしてみましょう。

In [ ]:
import cv2
import numpy as np
from scipy import misc
i = misc.ascent()


次に、pyplotライブラリを使って画像を描画することで、どのように見えるか確認しましょう。

In [ ]:
import matplotlib.pyplot as plt
plt.grid(False)
plt.gray()
plt.axis('off')
plt.imshow(i)
plt.show()

これは階段の吹き抜けのイメージであることがわかります。ここにはたくさんの特徴があるので、それらを分離できるかどうかを試すことができます -- 例えば、強い垂直線があります。

画像は numpy 配列として保存されているので、その配列をコピーするだけで変換された画像を作ることができます。画像の次元も取得しておきましょう。

In [ ]:
i_transformed = np.copy(i)
size_x = i_transformed.shape[0]
size_y = i_transformed.shape[1]

これでフィルタを 3x3 の配列として作成できるようになりました。

In [ ]:
# This filter detects edges nicely
# It creates a convolution that only passes through sharp edges and straight
# lines.

#Experiment with different values for fun effects.
#filter = [ [0, 1, 0], [1, -4, 1], [0, 1, 0]]

# A couple more filters to try for fun!
filter = [ [-1, -2, -1], [0, 0, 0], [1, 2, 1]]
#filter = [ [-1, 0, 1], [-2, 0, 2], [-1, 0, 1]]

#filter = [ [0, -1, 0], [-1, 5, -1], [0, -1, 0]]


# 下のセルで垂直線強調としているフィルタ例
#filter = [[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]]
# 下のセルで水平線強調としているフィルタ例
#filter = [[-1, -2, -1], [0, 0, 0], [1, 2, 1 ]]

# If all the digits in the filter don't add up to 0 or 1, you 
# should probably do a weight to get it to do so
# so, for example, if your weights are 1,1,1 1,2,1 1,1,1
# They add up to 10, so you would set a weight of .1 if you want to normalize them
weight  = 1

ここで畳み込みを作成してみましょう。1 ピクセルの余白を残して画像を繰り返し処理し、 現在の画素の隣り合う画素にフィルターで定義された値をかけます。（小林注：3x3のフィルタをパディングをあてずに畳み込みをすることを言っています）

つまり、 現在の画素のその上と左隣の画素にはフィルターの左上の項目が掛けられます。そしてその結果に重みをかけて 0 から 255 の範囲内に収まるようにします。

最後に、変換された画像に新しい値を読み込みます。

In [ ]:
for x in range(1,size_x-1):
  for y in range(1,size_y-1):
      convolution = 0.0
      convolution = convolution + (i[x - 1, y-1] * filter[0][0])
      convolution = convolution + (i[x, y-1] * filter[0][1])
      convolution = convolution + (i[x + 1, y-1] * filter[0][2])
      convolution = convolution + (i[x-1, y] * filter[1][0])
      convolution = convolution + (i[x, y] * filter[1][1])
      convolution = convolution + (i[x+1, y] * filter[1][2])
      convolution = convolution + (i[x-1, y+1] * filter[2][0])
      convolution = convolution + (i[x, y+1] * filter[2][1])
      convolution = convolution + (i[x+1, y+1] * filter[2][2])
      convolution = convolution * weight
      if(convolution<0):
        convolution=0
      if(convolution>255):
        convolution=255
      i_transformed[x, y] = convolution

これで、画像をプロットして畳み込みの効果を確認することができます!

In [ ]:
# Plot the image. Note the size of the axes -- they are 512 by 512
plt.gray()
plt.grid(False)
plt.imshow(i_transformed)
#plt.axis('off')
plt.show()   

そこで以下のフィルタの値とその画像への影響を考えてみましょう。

-1, 0, 1, -2, 0, 2, -1, 0, 1 を使うと非常に強い垂直線の集合が得られます。

![Detecting vertical lines filter](https://storage.googleapis.com/laurencemoroney-blog.appspot.com/MLColabImages/lab3-fig2.png)

-1, -2, -1, 0, 0, 0, 1, 2, 1 を使うと、水平線が得られます。

![Detecting horizontal lines](https://storage.googleapis.com/laurencemoroney-blog.appspot.com/MLColabImages/lab3-fig3.png)

違う値で試してみましょう！ 

## Pooling

畳み込みを使用するのと同様に、プーリングは特徴の検出に大きく役立ちます。目的は、検出された特徴をそのまま維持しながら、画像内の情報量を全体的に減らすことです。

プーリングにはいくつかの異なるタイプがありますが、ここでは MAX プーリングと呼ばれるものを使用します。

 ここでの考え方は、画像を繰り返し処理して、そのピクセルと、そのピクセルの右、下、右下ピクセルを見ていくことです。その中から最大のもの (これが MAX プーリングという名前の由来です) を取り出して新しい画像に読み込みます。このようにして、新しい画像は古い画像の 1/4 のサイズになります -- X と Y の寸法はこのプロセスによって半分になります。この圧縮にもかかわらず、画像の特徴が維持されていることがわかります。

![Max Pooling](https://storage.googleapis.com/laurencemoroney-blog.appspot.com/MLColabImages/lab3-fig4.png)

![CNN](https://jp.mathworks.com/help/deeplearning/ug/no_padding_no_strides.gif)

このコードは、(2, 2)プーリングを表示します。これを実行して出力を見てみると、画像のサイズが元の1/4であるにもかかわらず、抽出された特徴が維持されていることがわかるでしょう!


In [ ]:
new_x = int(size_x/2)
new_y = int(size_y/2)
newImage = np.zeros((new_x, new_y))
for x in range(0, size_x, 2):
  for y in range(0, size_y, 2):
    pixels = []
    pixels.append(i_transformed[x, y])
    pixels.append(i_transformed[x+1, y])
    pixels.append(i_transformed[x, y+1])
    pixels.append(i_transformed[x+1, y+1])
    pixels.sort(reverse=True)
    newImage[int(x/2),int(y/2)] = pixels[0]

# Plot the image. Note the size of the axes -- now 256 pixels instead of 512
plt.gray()
plt.grid(False)
plt.imshow(newImage)
#plt.axis('off')
plt.show()      
    
    

次では、Fashion MNISTニューラルネットワークに畳み込みを追加して、分類がより効率的に動作する（元画像のピクセルを利用するのではなく、特徴を利用して分類するため）ところを見てみましょう。